**without Pipeline**

In [5]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline


In [29]:
!pip install --upgrade scikit-learn

In [6]:
df = pd.read_csv('/content/Titanic-Dataset.csv')

In [7]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [9]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [10]:
# step 1 - > train test split
x = df.drop(columns=['Survived'],axis=1)
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [11]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [38]:
# applying imputation
from sklearn.impute import SimpleImputer
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])[:, 0]  # Extract the first (and only) column
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])[:, 0]  # Extract the first (and only) column

x_test_age = si_age.fit_transform(x_test[['Age']])[:, 0]  # Extract the first (and only) column
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])[:, 0]  # Extract the first (and only) column

In [31]:
from sklearn.preprocessing import OneHotEncoder

# Onehot encoding on sex and embarked
ohe_sex = OneHotEncoder(handle_unknown='ignore') # handle_unknown='ignore' will ignore unknown categories during transform
ohe_embarked = OneHotEncoder(handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']]).toarray()[:, 0]  # Extract the first (and only) column and convert to dense array
x_train_embarked = ohe_embarked.fit_transform(x_train[['Embarked']]).toarray()[:, 0]  # Extract the first (and only) column and convert to dense array

x_test_sex = ohe_sex.transform(x_test[['Sex']]).toarray()[:, 0]  # Extract the first (and only) column and convert to dense array
x_test_embarked = ohe_embarked.transform(x_test[['Embarked']]).toarray()[:, 0]  # Extract the first (and only) column and convert to dense array

In [32]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [33]:
X_train_rem = x_train.drop(columns=['Sex','Embarked'],axis=1)
X_test_rem = x_test.drop(columns=['Sex','Embarked'],axis=1)

In [34]:
X_train_transformed = np.concatenate((X_train_rem,
                                     x_train_age.reshape(-1, 1),  # Reshape to 2D
                                     x_train_sex.reshape(-1, 1),   # Reshape to 2D
                                     x_train_embarked.reshape(-1, 1)), axis=1)  # Reshape to 2D

X_test_transformed = np.concatenate((X_test_rem,
                                    x_test_age.reshape(-1, 1),   # Reshape to 2D
                                    x_test_sex.reshape(-1, 1),    # Reshape to 2D
                                    x_test_embarked.reshape(-1, 1)), axis=1)   # Reshape to 2D

In [35]:
X_train_transformed.shape

(712, 8)

In [39]:
lr = LogisticRegression()
lr.fit(X_train_transformed,y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
y_pred = lr.predict(X_test_transformed)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [15]:
import pickle
pickle.dump(ohe_sex,open('ohe_sex.pkl','wb'))
pickle.dump(ohe_embarked,open('ohe_embarked.pkl','wb'))
pickle.dump(lr,open('lr.pkl','wb'))

NameError: name 'ohe_sex' is not defined

**with pipeline**

In [40]:
# plan
# 1. Impute
# 2. One hot encoding
# 3. Scalling
# 4. Feature selection
# 5. model training

In [41]:
df = pd.read_csv('/content/Titanic-Dataset.csv')
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

# step 1 - > train test split
x = df.drop(columns=['Survived'],axis=1)
y = df['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [42]:
# step 1 Imputation transformer
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), # 2 is the column index
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6]) # 6 is the column index
],remainder='passthrough') # passthough is for remaining columns

In [53]:
# one hot encoding
from sklearn.preprocessing import OneHotEncoder

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(handle_unknown='ignore'),[1,6]) # Use handle_unknown='ignore' to handle unknown categories
],remainder='passthrough')

In [45]:
# scaling
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10)) # (0,10) means from 0 to 10 column
])

In [46]:
# feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
trf4 = SelectKBest(score_func=chi2,k=8) # k is the number of columns

In [47]:
# training
trf5 = LogisticRegression()

# Create Pipeline

In [48]:
from sklearn.pipeline import Pipeline

In [49]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5)
])

# Pipeline vs make_pipeline

pipeline requires naming of steps, make_pipeline does not .

(same applies to coloumnTransformer vs make_column_transformer)

In [50]:
# alternative
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)




In [52]:
# training
pipe.fit(x_train,y_train)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

# Explore Pipeline

# Cross validation using pipeline

In [54]:
# cross validation
from sklearn.model_selection import cross_val_score
score = cross_val_score(pipe,x_train,y_train,cv=5,scoring='accuracy').mean()


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/compose/_column_transformer.py", line 1110, in _hstack
    converted_Xs = [
  File "/usr/local/lib/python3.10/dist-packages/sklearn/compose/_column_transformer.py", line 1111, in <listcomp>
    check_array(X, accept_sparse=True, force_all_finite=False)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py", line 1012, in check_array
    array = _asarray_with_order(array, order=order, dtype=dtype, xp=xp)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py", line 751, in _asarray_with_order
    array = numpy.asarray(array, order=order, dtype=d

# GridSearch using Pipeline

In [26]:
# grid search
from sklearn.model_selection import GridSearchCV
# Assuming 'logisticregression' is the name of your final estimator in the pipeline
params = {
    'logisticregression__C':[0.1,1,10,100],
    'logisticregression__penalty':['l2'] # Change the penalty to 'l2'
}
grid = GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(x_train,y_train)
grid.best_score_
grid.best_params_

NameError: name 'pipe' is not defined

In [ ]:
pip install sklearn

In [27]:
# update sklearn
!pip install --upgrade scikit-learn